Some pre-requisites and things I had to install, mostly written here so I don't forget and that they're all in one place

In [3]:
!pip install -U torch -q #pytorch 
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 #this version with CUDA required for gemma
!pip install -U transformers datasets -q #huggingface transformers and datasets
!pip install -U accelerate -q #required to make gemma work

In [1]:
import torch
torch.cuda.is_available()

True

## Generate a sentence containing a given word

### English with gemma 2b (instruct)

In [14]:
#this model is gated, so... logging in to huggingface with #huggingface-cli login# is necessary
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")



Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
input_text = "Write a sentence that contains the word 'dancing'." #...it's the same everytime x_x
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=100)
print(tokenizer.decode(outputs[0]))

<bos>Please write me a sentence that contains the word 'dancing'.

The dancing flames illuminated the night sky, casting a mesmerizing glow on the bustling city streets.<eos>


### English with Mistral 7B Instruct v0.2

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map="auto")

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\accelerate\utils\modeling.py:1363: UserWarning: Current model requires 905976576 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
human_prompt = "Write me a sentence containing the word 'dancing'."
prompt = f"<s>[INST]{human_prompt}[/INST]"

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.9,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Write me a sentence containing the word 'dancing'.[/INST] Under the twinkling stars, a group of children laughed and danced together, their joyful movements illuminating the night.


Samples:

#this one took 14 minutes to generate x_x Completely unacceptable...
[INST]Write me a sentence containing the word 'dancing'.[/INST] Under the twinkling stars, a group of friends laughed and danced together, creating unforgettable memories at the outdoor summer party.

#this time it took 2.5 min for some reason, which is not great but fine I guess... except...
Under the twinkling stars, a group of friends let go of their worries and danced joyfully to the rhythm of the lively music.

#ok what is up with these twinkling stars? x_x
Under the twinkling stars, a group of friends laughed and danced together, creating a joyful and unforgettable moment.

#changed temperature from 0.1 to 0.5, let's see if it's any different
Under the twinkling fairy lights, a group of friends laughed and cheered as they danced together, lost in the joyful rhythm of the music.
#xDDDDDD

#HOTTER! temperature 0.9!!
Amidst the lively tunes of the marimba band, everyone was dancing joyfully in the moonlit garden.

#changed nothing and we're back to the twinkling stars xD
Under the twinkling stars, a group of children laughed and danced together, their joyful movements illuminating the night.





### English with quantized Mistral 7B Instruct v0.2 (https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF)

In [11]:
!pip3 install huggingface-hub
!pip install llama-cpp-python
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

.\mistral-7b-instruct-v0.2.Q4_K_M.gguf


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf to C:\Users\chase\.cache\huggingface\hub\tmpvlgzelvr


In [23]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
    model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",  # Download the model file first
    #chat_format= "llama-2", # The chat format to use - either "llama-1" or "llama-2" -- CAN BE OMITTED FOR DEFAULT FLAVOR
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

In [30]:
prompt = "Write me a sentence containing the word 'dances'."

# Simple inference example
output = llm(
    f"<s>[INST] {prompt} [/INST]", # Prompt
    max_tokens=120,  # Generate up to 120 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=True        # Whether to echo the prompt
)
#print(output)
print(output["choices"][0]["text"])


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1745.18 ms
llama_print_timings:      sample time =       4.79 ms /    33 runs   (    0.15 ms per token,  6886.48 tokens per second)
llama_print_timings: prompt eval time =    1034.07 ms /    13 tokens (   79.54 ms per token,    12.57 tokens per second)
llama_print_timings:        eval time =    5527.43 ms /    32 runs   (  172.73 ms per token,     5.79 tokens per second)
llama_print_timings:       total time =    6652.09 ms /    45 tokens


<s>[INST] Write me a sentence containing the word 'dances'. [/INST] The colorful performers gracefully danced to the lively rhythm of the traditional music, mesmerizing the crowd with their elegant movements and synchronized steps.


Samples:

[INST]Write me a sentence containing the word 'dancing'.
Amidst the lively melody, people of all ages were dancing joyfully together at the outdoor festival.
Amidst the lively tunes of the mariachi band, everyone was dancing and laughing, their joyful spirits intertwined in the warm summer evening.
Under the twinkling stars, they danced together in perfect harmony, their laughter echoing through the night. #NOOOOOOOO T-T
As the sun began to set, the couple lost themselves in the rhythm of the music and started dancing passionately under the twinkling stars. #IT'S BACK, AT THE BACK
As the sun began to set, the crowd at the festival came alive with excitement as they formed a circle and started dancing together, lost in the rhythm of the lively music playing from the speakers.
As the music played, everyone got up from their seats and started dancing enthusiastically, lost in the joy of the moment.
Under the twinkling stars, they all gathered around the bonfire, laughing and clapping as they watched the graceful ballerinas perform enchanting dances at the village festival.

#ok so these are very similar except they take 5 seconds or less, which is obviously strictly better, we'll go with this one after all

#also if we change the word at all, we get different results, so that's good too. Something to keep in mind for sure.
[INST] Write me an example sentence containing the word 'dance'. [/INST] 
On the vibrant and colorful stage, the ballerinas danced gracefully to the enchanting melodies of Tchaikovsky's Swan Lake.
Sarah twirled gracefully across the floor as she danced to the lively rhythm of the salsa music at the local community center.
[INST] Write me an example sentence containing the word 'dances'. [/INST] At the annual village festival, children of all ages danced joyfully around the maypole, weaving colorful ribbons in a lively and synchronous pattern.
The colorful performers gracefully danced to the lively rhythm of the traditional music, mesmerizing the crowd with their elegant movements and synchronized steps.


### Polish with Bielik 7B Instruct v0.1 GPTQ (quantized)

In [5]:
%%capture

!pip install optimum -q
!pip install auto-gptq -q
!pip install ctransformers[cuda]

In [6]:
import gc
import torch
import warnings

#warnings.filterwarnings("ignore") # good for demo, not good idea for production :)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "speakleash/Bielik-7B-Instruct-v0.1-GPTQ" #https://colab.research.google.com/drive/1Al9glPVCuOXbtDsks8cMcuzkuu8YDzpg?usp=sharing#scrollTo=LDo7PEL-gyyy
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')
gptq=True

CUDA extension not installed.
CUDA extension not installed.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chase\.cache\huggingface\hub\models--speakleash--Bielik-7B-Instruct-v0.1-GPTQ. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\transformers\modeling_utils.p

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [4]:
human_prompt = "Proszę napisać mi zdanie zawierające słowo 'taniec'" #albo tańczący
prompt = f"<s>[INST]{human_prompt}[/INST]"
#print(prompt)

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Proszę napisać mi zdanie zawierające słowo 'taniec'[/INST]  Taniec to wyjątkowa forma ekspresji, która łączy w sobie ruch, muzykę i emocje.


In [10]:
human_prompt = ("Napisz zdanie zawierające słowo 'tańczący'.") 
prompt = f"<s>[INST]{human_prompt}[/INST]"
#print(prompt)

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Napisz zdanie zawierające słowo 'tańczący'.[/INST]  W parku można było zobaczyć grupę tańczących par, które spędzały miło czas na świeżym powietrzu.


In [13]:
human_prompt = ("Napisz 4 przykładowe zdania, w ktorych się pojawia słowo 'taniec'.")
prompt = f"<s>[INST]{human_prompt}[/INST]"
#print(prompt)

outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\chase\anaconda3\envs\untitled\lib\logging\__init__.py", line 1085, in emit
    msg = self.format(record)
  File "C:\Users\chase\anaconda3\envs\untitled\lib\logging\__init__.py", line 929, in format
    return fmt.format(record)
  File "C:\Users\chase\anaconda3\envs\untitled\lib\logging\__init__.py", line 668, in format
    record.message = record.getMessage()
  File "C:\Users\chase\anaconda3\envs\untitled\lib\logging\__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\chase\anaconda3\envs\untitled\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\chase\anaconda3\envs\untitled\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\chase\anaconda3\envs\untitled\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
 

<s>[INST]Napisz 4 przykładowe zdania, w ktorych się pojawia słowo 'taniec'.[/INST]  Oto przykładowe zdania, w których pojawia się słowo "taniec":

1. Taniec to jedna z najstarszych form sztuki, wyrażająca radość i wolność poprzez ruch.
2. Uwielbiam patrzeć, jak moje dzieci tańczą bez tremy, wyrażając siebie w sposób naturalny.
3. Taniec towarzyski to forma sztuki, która łączy partnerów w harmonijnym ruchu i muzyce.
4. Taniec brzucha to technika, która łączy w sobie elementy akrobatyki, izraelskiego folkloru i jogi, wyrażając wewnętrzne piękno kobiecości.


Samples:

[INST]Napisz 4 przykładowe zdania, w ktorych znajduje się słowo 'taniec'.[/INST]  Oto przykłady zdań, w których pojawia się słowo "taniec":

1. Taniec to wyrafinowana forma sztuki, która łączy w sobie ruch, muzykę i emocje.
2. Julia pokochała taniec od pierwszego wejrzenia, gdy tylko zobaczyła transmisję z baletu.
3. W weekendowym parku ludzie tańczyli wokół ognia, świętując nadejście wiosny.
4. Taniec towarzyski stał się jego pasją, odkąd zaczął brać lekcje i wygrywać turnieje.

[INST]Napisz 4 przykładowe zdania, w ktorych się pojawia słowo 'taniec'.[/INST]  Oto przykładowe zdania, w których pojawia się słowo "taniec":

1. Taniec to jedna z najstarszych form sztuki, wyrażająca radość i wolność poprzez ruch.
2. Uwielbiam patrzeć, jak moje dzieci tańczą bez tremy, wyrażając siebie w sposób naturalny.
3. Taniec towarzyski to forma sztuki, która łączy partnerów w harmonijnym ruchu i muzyce.
4. Taniec brzucha to technika, która łączy w sobie elementy akrobatyki, izraelskiego folkloru i jogi, wyrażając wewnętrzne piękno kobiecości.